# Test with querying narrower concepts

First we import SPARQLWrapper

In [103]:
import SPARQLWrapper

Then we define what concept we're interested in

In [104]:
concept = "http://stad.gent/id/concepts/decision_making_themes/concept_20"

Then we can create our endpoint and query

In [105]:
stadgent = SPARQLWrapper.SPARQLWrapper("https://stad.gent/sparql")
stadgent.setReturnFormat(SPARQLWrapper.JSON)

Now we can wrap the querying in a function, so we can easily call it repeatedly

In [106]:
def get_narrower(concept):
    query = """
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

        SELECT *
        WHERE {
        <%s>    skos:prefLabel ?label;
                skos:narrower ?narrower.
        ?narrower skos:prefLabel ?narrowerLabel .
        }
    """ % concept

    stadgent.setQuery(query)

    results = stadgent.query().convert()

    narrower = {}

    for result in results["results"]["bindings"]:
        narrower[result["narrower"]["value"]] = result["narrowerLabel"]["value"]
    
    if len(narrower) == 0:
        return None
    return narrower

Now we can run the function, and get the first concept

In [107]:
concepts = get_narrower(concept)

print(concepts)

{'http://stad.gent/id/concepts/decision_making_themes/concept_21': 'Parkeren', 'http://stad.gent/id/concepts/decision_making_themes/concept_28': 'Openbare weg en pleinen', 'http://stad.gent/id/concepts/decision_making_themes/concept_29': 'Nutsvoorzieningen', 'http://stad.gent/id/concepts/decision_making_themes/concept_30': 'Mobiliteit', 'http://stad.gent/id/concepts/decision_making_themes/concept_34': 'Straatnaamgeving', 'http://stad.gent/id/concepts/decision_making_themes/concept_35': 'Straatverlichting', 'http://stad.gent/id/concepts/decision_making_themes/concept_36': 'Wegenregister'}


After we've now gotten the first level, we can use a recursive function to get all the levels

In [108]:
def get_all_narrower(concept):
    narrower = get_narrower(concept)
    if narrower == None:
        return
    for narrower_concept in narrower:
        concepts[narrower_concept] = narrower[narrower_concept]
        get_all_narrower(narrower_concept)

Now we can run the recursive function, and get all the levels and print them out

In [109]:
get_all_narrower(concept)

print(concepts)

{'http://stad.gent/id/concepts/decision_making_themes/concept_21': 'Parkeren', 'http://stad.gent/id/concepts/decision_making_themes/concept_28': 'Openbare weg en pleinen', 'http://stad.gent/id/concepts/decision_making_themes/concept_29': 'Nutsvoorzieningen', 'http://stad.gent/id/concepts/decision_making_themes/concept_30': 'Mobiliteit', 'http://stad.gent/id/concepts/decision_making_themes/concept_34': 'Straatnaamgeving', 'http://stad.gent/id/concepts/decision_making_themes/concept_35': 'Straatverlichting', 'http://stad.gent/id/concepts/decision_making_themes/concept_36': 'Wegenregister', 'http://stad.gent/id/concepts/decision_making_themes/concept_22': 'Bewonersparkeren', 'http://stad.gent/id/concepts/decision_making_themes/concept_23': 'Park and ride', 'http://stad.gent/id/concepts/decision_making_themes/concept_24': 'Parkeergarages', 'http://stad.gent/id/concepts/decision_making_themes/concept_25': 'Parkeerretributies en parkeerboetes', 'http://stad.gent/id/concepts/decision_making_t